In [1]:
class IStrategy :

    def __init__(self, name="IStrategy"):
        self.name = name

    def isRandom(self):
        return False

    def getAction(self, game, unityPrice, date, debug=False):
        '''
        @return (action, amountOfSecurity)
        '''
        pass

    def getName(self):
        '''
        @return the name of the strategy
        '''
        return self.name

In [2]:
class StrategyLongTerm(IStrategy):
    def __init__(self, dateAchat, dateVente):
        super().__init__("LongTerm")
        self.dateAchat=dateAchat
        self.dateVente=dateVente
        self.nb_and_cost_Sell=["nb_Sell:",0,"cost_Sell",0]
        self.nb_and_cost_Buy=["nb_Buy:",0,"cost_Buy",0]
        self.nb_and_cost_Still=["nb_Still:",0,"cost_Still",0]

    def getAction(self, game, unityPrice, date, debug=False):
        if date==self.dateAchat:
            self.nb_and_cost_Buy[1]+=1
            self.nb_and_cost_Buy[3]+=game.account.cash/unityPrice
            return (Game.BUY,(game.account.cash/unityPrice))

        elif (date==self.dateVente):
            self.nb_and_cost_Sell[1]+=1
            self.nb_and_cost_Sell[3]+=game.account.security
            return (Game.SELL, game.account.security)

        elif (date<self.dateAchat) or (date>self.dateVente):
            self.nb_and_cost_Still[1]+=1
            return (Game.STILL, 0)

        else:
            self.nb_and_cost_Still[1]+=1
            return (Game.STILL, 0)

In [3]:
class StrategyRandom(IStrategy):
    def __init__(self):
        super().__init__("Random")
        self.nb_and_cost_Sell=["nb_Sell:",0,"cost_Sell",0]
        self.nb_and_cost_Buy=["nb_Buy:",0,"cost_Buy",0]
        self.nb_and_cost_Still=["nb_Still:",0,"cost_Still",0]

    def getAction(self, game, unityPrice, date, debug=False):
        choice = random.randint(0, 2)
        value_cash = game.account.cash*random.random()
        value_sell = game.account.security*random.random()

        if choice==0:
            self.nb_and_cost_Buy[1]+=1
            self.nb_and_cost_Buy[3]+=(value_cash/unityPrice)
            return (Game.BUY, value_cash/unityPrice)
        elif choice==1:
            self.nb_and_cost_Sell[1]+=1
            self.nb_and_cost_Sell[3]+=value_sell
            return (Game.SELL, value_sell)
        else:
            self.nb_and_cost_Still[1]+=1
            return (Game.STILL, 0)

In [4]:
class StrategyOracle(IStrategy):
    #le paramètre previsionJour permet de décider sur combien de jours ce font les prévisions
    def __init__(self, previsionJour=1):
        super().__init__("Oracle")
        self.previsionJour=previsionJour
        self.nb_and_cost_Sell=["nb_Sell:",0,"cost_Sell",0]
        self.nb_and_cost_Buy=["nb_Buy:",0,"cost_Buy",0]
        self.nb_and_cost_Still=["nb_Still:",0,"cost_Still",0]

    def getAction(self, game, unityPrice, date, debug=False):
        if len(game.prices) > game.clock + self.previsionJour :
            futurPrice = game.prices[game.clock + self.previsionJour]
            if debug:
                print(futurPrice)
            if (futurPrice>unityPrice):
                return (Game.BUY, (game.account.cash/unityPrice))
            elif (futurPrice<unityPrice):
                self.nb_and_cost_Sell[1]+=1
                self.nb_and_cost_Sell[3]+=game.account.security
                return (Game.SELL, (game.account.security))
            else:
                self.nb_and_cost_Still[1]+=1
                return (Game.STILL, (game.account.security))
        else:
            self.nb_and_cost_Still[1]+=1
            return (Game.STILL, (game.account.security))

In [5]:
class StrategyEffetJanvier(IStrategy):
    def __init__(self):
        super().__init__("EffetJanvier")
        self.nb_and_cost_Sell=["nb_Sell:",0,"cost_Sell",0]
        self.nb_and_cost_Buy=["nb_Buy:",0,"cost_Buy",0]
        self.nb_and_cost_Still=["nb_Still:",0,"cost_Still",0]

    def getAction(self, game, unityPrice, date, debug=False):
        if date.month == 1 :
            self.nb_and_cost_Buy[1]+=1
            self.nb_and_cost_Buy[3]+=game.account.cash/unityPrice
            return (Game.BUY, (game.account.cash/unityPrice))
        elif date.month == 2 :
            self.nb_and_cost_Sell[1]+=1
            self.nb_and_cost_Sell[3]+=game.account.security
            return (Game.SELL, (game.account.security))
        else :
            self.nb_and_cost_Still[1]+=1
            return (Game.STILL, game.account.security)

In [6]:
class StrategyKinjun_Sen(IStrategy):

    def __init__(self, kijun_period=26):
        super().__init__(name="Kinjun_Sen")
        self.kijun_period = kijun_period
        self.nb_and_cost_Sell=["nb_Sell:",0,"cost_Sell",0]
        self.nb_and_cost_Buy=["nb_Buy:",0,"cost_Buy",0]
        self.nb_and_cost_Still=["nb_Still:",0,"cost_Still",0]

    def getAction(self, game, unityPrice, date, debug=False):
        #regarde si l'historique des prix contient suffisamment de données pour calculer la moyenne Kinjun, soit jusqu'au 26ème jour.
        if game.clock < self.kijun_period:
            return (Game.STILL, game.account.security)

        kinjun_max=max(game.prices[game.clock-self.kijun_period:game.clock])
        kinjun_min=min(game.prices[game.clock-self.kijun_period:game.clock])

        #moyenne entre le plus hauts et les plus bas prix de cette période
        kijun_moy = (kinjun_max + kinjun_min) / 2

        if unityPrice > kijun_moy:
            self.nb_and_cost_Buy[1]+=1
            self.nb_and_cost_Buy[3]+=game.account.cash/unityPrice

            return (Game.BUY, game.account.cash / unityPrice)

        elif unityPrice < kijun_moy:
            self.nb_and_cost_Sell[1]+=1
            self.nb_and_cost_Sell[3]+=game.account.security

            return (Game.SELL, game.account.security)

        else:
            self.nb_and_cost_Still[1]+=1

            return (Game.STILL, game.account.security)

In [7]:
class StrategyDollarSpendOverTime(IStrategy):
    def __init__(self, amount_to_spend=5):
        super().__init__("DollarSpendOverTime")
        self.amount_to_spend = amount_to_spend
        self.nb_and_cost_Sell=["nb_Sell:",0,"cost_Sell",0]
        self.nb_and_cost_Buy=["nb_Buy:",0,"cost_Buy",0]
        self.nb_and_cost_Still=["nb_Still:",0,"cost_Still",0]

    def getAction(self, game, unityPrice, date, debug=False):
        if self.amount_to_spend > game.account.cash:
            self.nb_and_cost_Still[1]+=1
            return (Game.STILL, 0)
        elif self.amount_to_spend <= game.account.cash:
            self.nb_and_cost_Buy[1]+=1
            self.nb_and_cost_Buy[3]+=game.account.cash/unityPrice
            return (Game.BUY, self.amount_to_spend/unityPrice)

In [8]:
class StrategyCoinFlip(IStrategy):
    def __init__(self):
        super().__init__("CoinFlip")
        self.nb_and_cost_Sell=["nb_Sell:",0,"cost_Sell",0]
        self.nb_and_cost_Buy=["nb_Buy:",0,"cost_Buy",0]
        self.nb_and_cost_Still=["nb_Still:",0,"cost_Still",0]

    def getAction(self, game, unityPrice, date, debug=False):

        pile_face = random.randint(0, 1)

        if pile_face == 0:
            shares_to_buy = game.account.cash / unityPrice
            if shares_to_buy > 0:
                self.nb_and_cost_Buy[1]+=1
                self.nb_and_cost_Buy[3]+=game.account.cash/unityPrice
                return (Game.BUY, shares_to_buy)
        else:
            if game.account.security > 0:
                self.nb_and_cost_Sell[1]+=1
                self.nb_and_cost_Sell[3]+=game.account.security
                return (Game.SELL, game.account.security)
        self.nb_and_cost_Still[1]+=1
        return (Game.STILL, 0)

In [9]:
class StrategyVeille(IStrategy):
    def __init__(self):
        super().__init__("Veille")
        self.nb_and_cost_Sell=["nb_Sell:",0,"cost_Sell",0]
        self.nb_and_cost_Buy=["nb_Buy:",0,"cost_Buy",0]
        self.nb_and_cost_Still=["nb_Still:",0,"cost_Still",0]

    def getAction(self, game, unityPrice, date, debug=False):

        veille = game.prices[game.clock-1]

        if veille < unityPrice:
            shares_to_buy = game.account.cash / unityPrice
            self.nb_and_cost_Buy[1]+=1
            self.nb_and_cost_Buy[3]+=shares_to_buy
            return (Game.BUY, shares_to_buy)
        elif veille > unityPrice :
            self.nb_and_cost_Sell[1]+=1
            self.nb_and_cost_Sell[3]+=game.account.security
            return (Game.SELL, game.account.security)
        else :
            self.nb_and_cost_Still[1]+=1
            return (Game.STILL, 0)

In [10]:
#strategy Les investisseurs doivent être prêts à vendre des actions si elles ne répondent plus à leurs critères d'investissement ou si une meilleure opportunité d'investissement se présente.
class StrategyInvestmentThreshold(IStrategy):

    def __init__(self, threshold):
        super().__init__(name="InvestmentThreshold")
        #self.serie = serie
        self.threshold = threshold
        self.nb_and_cost_Sell=["nb_Sell:",0,"cost_Sell",0]
        self.nb_and_cost_Buy=["nb_Buy:",0,"cost_Buy",0]
        self.nb_and_cost_Still=["nb_Still:",0,"cost_Still",0]

    def getAction(self, game, unityPrice, date, debug=False):

        if game.clock < 30 :
            return (Game.STILL, 0)

        #On calcule la moyenne mobile sur les 30 derniers instants
        #rolling = self.serie.rolling(30).mean()
        rollingmean = game.prices[game.clock-30:game.clock].mean()

        # On prend la dernière observation, observation du jour
        last_price = unityPrice

        # Si le dernier prix est en dessous du seuil fixé par l'utilisateur et que la moyenne mobile est également en dessous de ce seuil, 
        # alors on estime que le prix va remonter, donc on achète
        if last_price < self.threshold and rollingmean < self.threshold:
            self.nb_and_cost_Buy[1]+=1
            self.nb_and_cost_Buy[3]+=game.account.cash / last_price
            return (Game.BUY, game.account.cash / last_price)

        # Si le dernier prix est au-dessus du seuil fixé par l'utilisateur et que la moyenne mobile est également au-dessus de ce seuil,
        # alors on estime que le prix va baisser, donc on vend
        elif last_price > self.threshold and rollingmean > self.threshold:
            self.nb_and_cost_Sell[1]+=1
            self.nb_and_cost_Sell[3]+=game.account.security
            return (Game.SELL, game.account.security)

        # Sinon, on ne fait rien
        else:
            self.nb_and_cost_Still[1]+=1
            return (Game.STILL, 0)


In [11]:
class StrategyIntrinsicPrice(IStrategy):

    def __init__(self, margin_of_safety=0.1, period_accuracy=30):
        super().__init__(name="IntrinsicPrice")
        self.margin_of_safety = margin_of_safety
        self.period_accuracy = period_accuracy
        self.nb_and_cost_Sell=["nb_Sell:",0,"cost_Sell",0]
        self.nb_and_cost_Buy=["nb_Buy:",0,"cost_Buy",0]
        self.nb_and_cost_Still=["nb_Still:",0,"cost_Still",0]

    def getAction(self, game, unityPrice, date, debug=False):

        if game.clock < self.period_accuracy:
            return (Game.STILL, 0)

        #période sur laquelle on va pouvoir récupérer les bénéfices
        period_profit = game.prices[max(0, game.clock-self.period_accuracy):game.clock]

        # calculer le bénéfice moyen sur la période considérée
        avg_earnings = 0.0
        if len(period_profit) > 0:
            avg_earnings = sum(period_profit) / len(period_profit)
        else:
            avg_earnings = game.prices[game.clock]

        #calculer le ratio cours/bénéfice de la période considérée
        #le ratio reflète la valeur intrinsèque de la cryptomonnaie sans marge de sécurité
        ratio = unityPrice / avg_earnings

        #Récupérer le prix intrinsèque (intrinsic_price) en utilisant une marge de sécurité
        intrinsic_price = ratio * (1.0 - self.margin_of_safety)

        #Acheter si le prix est inférieur à intrinsic_price
        if unityPrice < intrinsic_price:
            value_buy = game.account.cash / unityPrice
            self.nb_and_cost_Buy[1]+=1
            self.nb_and_cost_Buy[3]+=value_buy
            return (Game.BUY, value_buy)

        #Vendre si le prix est supérieur à intrinsic_price
        elif unityPrice > intrinsic_price and game.account.security > 0:
            self.nb_and_cost_Sell[1]+=1
            self.nb_and_cost_Sell[3]+=game.account.security
            return (Game.SELL, game.account.security)

        #Rester sur place si le prix est égal à intrinsic_price
        else:
            self.nb_and_cost_Still[1]+=1
            return (Game.STILL, 0)